<a href="https://colab.research.google.com/github/cammylexi/CS2341-Assignment-3/blob/main/McPhaul_Llanes_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparation



https://www.kaggle.com/datasets/sunilthite/text-document-classification-dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [ ]:
import pandas as pd

df = pd.read_csv(path + "/IMDB Dataset.csv")

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Class Variables
Binary Classification Task


*   Sentiment
 *  Positive
 *  Negative

In [ ]:
# Change 'positive' to 1 and 'negative' to 0 in the 'sentiment' column
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
from transformers import BertTokenizer
import pandas as pd

# Load BERT tokenizer (lowercased version)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocess function using the tokenizer
def preprocess_text_bert(texts, max_length=256):
    return tokenizer(
        texts.tolist(),  # list of strings
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='tf',          # returns TensorFlow tensors
        return_attention_mask=True,   # include attention mask
        return_token_type_ids=False   # not needed for single-sentence inputs
    )

encoded_inputs = preprocess_text_bert(df['review'])

# You now have:
# - encoded_inputs['input_ids']: token ID sequences
# - encoded_inputs['attention_mask']: attention masks


import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((
    dict(input_ids=encoded_inputs['input_ids'],
         attention_mask=encoded_inputs['attention_mask']),
    df['sentiment'].map({'positive': 1, 'negative': 0})  # Adjust target labels
))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Stratified Ten Fold Cross Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import tensorflow as tf

# Assuming you've already preprocessed reviews and have these:
input_ids = encoded_inputs['input_ids'].numpy()
attention_mask = encoded_inputs['attention_mask'].numpy()
labels = df['sentiment'].map({'positive': 1, 'negative': 0}).values

# Set up Stratified K-Fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

fold = 1
for train_index, val_index in skf.split(input_ids, labels):
    print(f"Training Fold {fold}...")
    fold += 1

    # Split the data
    train_inputs = {
        'input_ids': input_ids[train_index],
        'attention_mask': attention_mask[train_index]
    }
    val_inputs = {
        'input_ids': input_ids[val_index],
        'attention_mask': attention_mask[val_index]
    }
    train_labels = labels[train_index]
    val_labels = labels[val_index]

    # Create datasets
    train_ds = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).batch(32).shuffle(1000)
    val_ds = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels)).batch(32)

    # Now train your model on train_ds and validate on val_ds...


ValueError: Input y contains NaN.